In [1]:
import transform
import scihub
import os
from dbutils import get_new_db_connection
from sentence_transformers import SentenceTransformer
import pandas as pd

In [2]:
conn, cur = get_new_db_connection()

In [3]:
results = cur.execute('''SELECT article FROM abstracts''')

In [4]:
results = cur.fetchall()
df = pd.DataFrame(results, columns=['article'])

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# tokenizer = AutoTokenizer.from_pretrained('ktrapeznikov/biobert_v1.1_pubmed_squad_v2')
# model = AutoModelForQuestionAnswering.from_pretrained('ktrapeznikov/biobert_v1.1_pubmed_squad_v2')

tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = AutoModelForQuestionAnswering.from_pretrained('dmis-lab/biobert-v1.1')

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Inference

In [6]:
import torch

question, text = "What is the study size?", "Although several studies have demonstrated that facial-affect recognition impairment is common following moderate-severe traumatic brain injury (TBI), and that there are diffuse alterations in large-scale functional brain networks in TBI populations, little is known about the relationship between the two. Here, in a sample of 26 participants with TBI and 20 healthy comparison participants (HC) we measured facial-affect recognition abil- ities and resting-state functional connectivity (rs-FC) using fMRI"
input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_ids = tokenizer.encode(input_text)
token_type_ids = [0 if i <= input_ids.index(102) else 1
                  for i in range(len(input_ids))]
start_scores, end_scores = model(torch.tensor([input_ids]),
                                 token_type_ids=torch.tensor([token_type_ids]))
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

. Here , in a sample of 26 participants with T ##BI and 20 healthy comparison participants ( HC ) we measured facial - affect recognition a ##bil - it ##ies and resting - state functional connectivity ( r ##s - FC ) using f ##MR ##I [SEP]


In [7]:
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

. Here , in a sample of 26 participants with T ##BI and 20 healthy comparison participants ( HC ) we measured facial - affect recognition a ##bil - it ##ies and resting - state functional connectivity ( r ##s - FC ) using f ##MR ##I [SEP]


In [8]:
df.to_csv('10k_articles.csv')


In [9]:
torch.tensor([input_ids])


tensor([[  101,   101,  1327,  1110,  1103,  2025,  2060,   136,   102,  1966,
          1317,  2527,  1138,  7160,  1115, 14078,   118,  6975,  4453, 24034,
          8341,  1880,  1110,  1887,  1378,  8828,   118,  5199, 23057,  3575,
          3773,   113,   157, 24963,   114,   117,  1105,  1115,  1175,  1132,
          4267,  3101,  5613, 18651,  1107,  1415,   118,  3418,  8458,  3575,
          6379,  1107,   157, 24963,  6623,   117,  1376,  1110,  1227,  1164,
          1103,  2398,  1206,  1103,  1160,   119,  3446,   117,  1107,   170,
          6876,  1104,  1744,  6635,  1114,   157, 24963,  1105,  1406,  8071,
          7577,  6635,   113, 18315,   114,  1195,  7140, 14078,   118,  6975,
          4453,   170, 15197,   118,  1122,  1905,  1105,  8137,   118,  1352,
          8458, 24466,   113,   187,  1116,   118,  3604,   114,  1606,   175,
         21148,  2240,   102,   102]])